In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# Acknowledgement to
# https://github.com/kuangliu/pytorch-cifar,
# https://github.com/BIGBALLON/CIFAR-ZOO,

''' Swish activation '''
class Swish(nn.Module): # Swish(x) = x∗σ(x)
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return input * torch.sigmoid(input)


class ConvNet(nn.Module):
    def __init__(self, channel, num_classes, net_width, net_depth, net_act, net_norm, net_pooling, im_size = (32,32)):
        super(ConvNet, self).__init__()

        self.features, shape_feat = self._make_layers(channel, net_width, net_depth, net_norm, net_act, net_pooling, im_size)
        num_feat = shape_feat[0]*shape_feat[1]*shape_feat[2]
        self.classifier = nn.Linear(num_feat, num_classes)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def embed(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        return out

    def _get_activation(self, net_act):
        if net_act == 'sigmoid':
            return nn.Sigmoid()
        elif net_act == 'relu':
            return nn.ReLU(inplace=True)
        elif net_act == 'leakyrelu':
            return nn.LeakyReLU(negative_slope=0.01)
        elif net_act == 'swish':
            return Swish()
        else:
            exit('unknown activation function: %s'%net_act)

    def _get_pooling(self, net_pooling):
        if net_pooling == 'maxpooling':
            return nn.MaxPool2d(kernel_size=2, stride=2)
        elif net_pooling == 'avgpooling':
            return nn.AvgPool2d(kernel_size=2, stride=2)
        elif net_pooling == 'none':
            return None
        else:
            exit('unknown net_pooling: %s'%net_pooling)

    def _get_normlayer(self, net_norm, shape_feat):
        # shape_feat = (c*h*w)
        if net_norm == 'batchnorm':
            return nn.BatchNorm2d(shape_feat[0], affine=True)
        elif net_norm == 'layernorm':
            return nn.LayerNorm(shape_feat, elementwise_affine=True)
        elif net_norm == 'instancenorm':
            return nn.GroupNorm(shape_feat[0], shape_feat[0], affine=True)
        elif net_norm == 'groupnorm':
            return nn.GroupNorm(4, shape_feat[0], affine=True)
        elif net_norm == 'none':
            return None
        else:
            exit('unknown net_norm: %s'%net_norm)

    def _make_layers(self, channel, net_width, net_depth, net_norm, net_act, net_pooling, im_size):
        layers = []
        in_channels = channel
        if im_size[0] == 28:
            im_size = (32, 32)
        shape_feat = [in_channels, im_size[0], im_size[1]]
        for d in range(net_depth):
            layers += [nn.Conv2d(in_channels, net_width, kernel_size=3, padding=3 if channel == 1 and d == 0 else 1)]
            shape_feat[0] = net_width
            if net_norm != 'none':
                layers += [self._get_normlayer(net_norm, shape_feat)]
            layers += [self._get_activation(net_act)]
            in_channels = net_width
            if net_pooling != 'none':
                layers += [self._get_pooling(net_pooling)]
                shape_feat[1] //= 2
                shape_feat[2] //= 2

        return nn.Sequential(*layers), shape_feat

In [ ]:
# Parameters
channel = 1
num_classes = 10
net_width = 64
net_depth = 3
net_act = 'relu'
net_norm = 'batchnorm'
net_pooling = 'maxpooling'
im_size = (28, 28) 

convNet3 = ConvNet(channel, num_classes, net_width, 3, net_act, net_norm, net_pooling, im_size)

In [ ]:
import torch
image_syn_eval_loaded, label_syn_eval_loaded = torch.load('image_syn_eval.pt')


C:\Users\Paul\AppData\Local\Temp\ipykernel_102732\3794286692.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image_syn_eval_loaded, label_syn_eval_loaded = torch.load('i

In [14]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(convNet3.parameters(), lr=0.001)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split
num_tasks = 5
task_datasets = random_split(TensorDataset(image_syn_eval_loaded, label_syn_eval_loaded), [len(image_syn_eval_loaded) // num_tasks] * num_tasks)
task_performance = []

In [ ]:
# Continual learning simulation
for task_id, task_data in enumerate(task_datasets):
    print(f"Training on task {task_id + 1}")
    task_loader = DataLoader(task_data, batch_size=32, shuffle=True)
    
    for epoch in range(5):
        for inputs, targets in task_loader:
            inputs = inputs.view(-1, channel, *im_size)
            optimizer.zero_grad()
            outputs = convNet3(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    task_accuracy = []
    with torch.no_grad():
        for seen_task in range(task_id + 1):
            seen_loader = DataLoader(task_datasets[seen_task], batch_size=32)
            correct, total = 0, 0
            for inputs, targets in seen_loader:
                inputs = inputs.view(-1, channel, *im_size)
                outputs = convNet3(inputs)
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
            accuracy = 100 * correct / total
            task_accuracy.append(accuracy)
        task_performance.append(task_accuracy)
        print(f"Task {task_id + 1} accuracies: {task_accuracy}")
        
torch.save({'task_performance': task_performance}, 'continual_learning_results.pt')

print("Continual learning simulation complete.")

Training on task 1
Task 1 accuracies: [100.0]
Training on task 2
Task 2 accuracies: [100.0, 100.0]
Training on task 3
Task 3 accuracies: [85.0, 85.0, 100.0]
Training on task 4
Task 4 accuracies: [80.0, 60.0, 95.0, 90.0]
Training on task 5
Task 5 accuracies: [85.0, 75.0, 75.0, 75.0, 70.0]
Continual learning simulation complete.
